# Paginacion

La definicion simple de paginacion es que cuando se intenta extaraer demaciada informacion, se divide dentro de multiples paginas.
Ejemplo: Amazon

La principal funcion de la paginacion es dividir mucha informacion en un formato muti pagina
Esto ayuda a reducir la carga en la API y tambien del cliente
En un supuesto de diez mil elementos al mismo tiempo o se pueden cargar 48 elementos al mismo tiempo. Se prefiere cargar 48 elementos al mismo tiempo.

Asi no se desperdician recursos mostrando demaciados resultados que pueden no ser relevantes y tal vez el usuario no llegue a verlos porque opta por las primeras opciones.

## Configuracion paginacion de Django REST framework

In [ ]:
https://www.django-rest-framework.org/api-guide/pagination/

In [ ]:
REST_FRAMEWORK = {
    'DEFAULT_PAGINATION_CLASS': 'rest_framework.pagination.LimitOffsetPagination',
    'PAGE_SIZE': 100
}

En el JSON es importante revisar los campos de:
- count
- next
- previous

Hay varios tipos de paginacion:
- PageNumberPagination
- LimitOffsetPagination
- CursorPagination


### PageNumberPagination

In [ ]:
Crear cierta cantidad de productos para experimentar con la paginacion (forms_test)
src/products2/pagination.py

In [ ]:
from rest_framework.pagination import PageNumberPagination

class ProductPagination(PageNumberPagination):
    #page_size = 4
    page_query_param = "p"
    page_size_query_param = "size"
    max_page_size = 5
    last_page_strings = "end"

In [ ]:
src/products2/serializers.py

In [ ]:
from rest_framework import serializers

from .models import Product

class ProductSerializer(serializers.ModelSerializer):
    class Meta:
        model = Product
        fields = ['user', 'title', 'slug']

In [ ]:
src/products2/views.py

In [ ]:
#...
from rest_framework import generics
from .pagination import ProductPagination
from .serializers import ProductSerializer
#...
class ProductListView(generics.ListAPIView):
    queryset = Product2.objects.all()
    serializer_class = ProductSerializer
    pagination_class = ProductPagination

### LimitOffsetPagination
- limit: tamaño de pagina
- offset: el numero de registros a saltar antes de seleccionar registros

Ejemplo
- 17 objetos en total
- limit 5
- offset 12
- Regresa los ultimos 5 elementos

In [ ]:
src/products2/pagination.py

In [ ]:
from rest_framework.pagination import PageNumberPagination, LimitOffsetPagination

class ProductPagination(PageNumberPagination):
    #page_size = 4
    page_query_param = "p"
    page_size_query_param = "size"
    max_page_size = 5
    last_page_strings = "end"

class ProductLOPagination(LimitOffsetPagination):
    defaut_limit = 5

In [ ]:
src/products2/views.py

In [ ]:
#...
from .pagination import ProductPagination, ProductLOPagination#<-----
#...
class ProductListView(generics.ListAPIView):
    queryset = Product2.objects.all()
    serializer_class = ProductSerializer
    pagination_class = ProductLOPagination#<-----

In [ ]:
src/products2/pagination.py

In [ ]:
from rest_framework.pagination import PageNumberPagination, LimitOffsetPagination

class ProductPagination(PageNumberPagination):
    #page_size = 4
    page_query_param = "p"
    page_size_query_param = "size"
    max_page_size = 5
    last_page_strings = "end"

class ProductLOPagination(LimitOffsetPagination):
    default_limit = 5
    max_limit = 7
    limit_query_param = "records"
    offset_query_param = "start"

### CursorPagination
- No se tiene numero de pagina
- Solo se tiene next y prevoius
- El orden por default es con un campo de created
    - creataed: desde el ultimo al primero

In [ ]:
src/products2/models.py

In [ ]:
from django.conf import settings
from django.db import models

User = settings.AUTH_USER_MODEL

class Product2(models.Model):
    user = models.ForeignKey(User, blank=True, null=True, on_delete=models.SET_NULL)
    title = models.CharField(max_length=120)
    slug = models.SlugField(unique=True)
    price = models.DecimalField(decimal_places=2, max_digits=20, default=10)
    created = models.DateTimeField(auto_now_add=True)#<-----

    def get_absolute_url(self):
        return f"/products/products/{self.slug}"
    
    def get_edit_url(self):
        return f"/products/my-products/{self.slug}"
    
    def get_delete_url(self):
        return f"/products/my-products/{self.slug}/delete"

class DigitalProduct2(Product2):
    class Meta:
        proxy = True

In [ ]:
python manage.py makemigrations
python manage.py migrate

In [ ]:
src/products2/pagination.py

In [ ]:
from rest_framework.pagination import PageNumberPagination, LimitOffsetPagination, CursorPagination

class ProductPagination(PageNumberPagination):
    #page_size = 4
    page_query_param = "p"
    page_size_query_param = "size"
    max_page_size = 5
    last_page_strings = "end"

class ProductLOPagination(LimitOffsetPagination):
    default_limit = 5
    max_limit = 7
    limit_query_param = "records"
    offset_query_param = "start"

class ProductCPagination(CursorPagination):
    page_size = 4

In [ ]:
src/products2/views.py

In [ ]:
#...
from .pagination import ProductPagination, ProductLOPagination, ProductCPagination#<-----
#...
class ProductListView(generics.ListAPIView):
    queryset = Product2.objects.all()
    serializer_class = ProductSerializer
    pagination_class = ProductCPagination#<-----

## Renderers - BrowsableAPI JSON
https://www.django-rest-framework.org/api-guide/renderers/

In [ ]:
src/config/settings.py

In [ ]:
#...
REST_FRAMEWORK = {
    "DEFAULT_RENDERER_CLASSES" : [
        "rest_framework.renderers.JSONRenderer"
    ]
}